In [ ]:
%load_ext autoreload
%autoreload 2
from typing import List, Optional
from omegaconf import OmegaConf
import os
import time
import json
import wandb
import logging
import argparse

import torch
from datasets.driving_dataset import DrivingDataset
from datasets.my_dataset import MyDataset
from utils.misc import import_str
from models.trainers import BasicTrainer
from models.video_utils import (
    render_images,
    save_videos,
    render_novel_views
)


In [ ]:
# 添加路径  ../data
cfg = OmegaConf.load(os.path.join("/mnt/e/Output/cam3/23", "config.yaml"))
cfg.data.pixel_source.cameras = [0, 3, 4]
cfg.data.pixel_source.downscale_when_loading = [2, 2, 2]
%cd /home/a/drivestudio

dataset = MyDataset(cfg.data)




In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainer = import_str(cfg.trainer.type)(
    **cfg.trainer,
    num_timesteps=dataset.num_img_timesteps,
    model_config=cfg.model,
    num_train_images=len(dataset.train_image_set),
    num_full_images=len(dataset.full_image_set),
    test_set_indices=dataset.test_timesteps,
    scene_aabb=dataset.get_aabb().reshape(2, 3),
    device=device

)

In [ ]:
trainer.resume_from_checkpoint(
    ckpt_path="/mnt/e/Output/cam3/23/checkpoint_20000.pth",
    load_only_model=True
)

In [ ]:
from utils.mytools import split_trajectory


traj = dataset.get_novel_render_traj(["front_center_interp"], dataset.frame_num)["front_center_interp"]
segments, ranges = split_trajectory(traj, min_count=5)
indices = []
for seg in segments:
    indices.append(3 * seg[-1] )
    indices.append(3 * seg[-1] + 1)
    indices.append(3 * seg[-1] + 2)
print(indices)

In [ ]:
results = render_images(trainer, dataset.full_image_set, vis_indices=indices)

In [ ]:
from matplotlib.pyplot import imshow
from utils.visualization import to8b
img = to8b(results["rgbs"][15])


In [ ]:
import cv2
cv2.imwrite("/mnt/e/Output/cam3/23/0.png",img)